#Introduction to Data Science with Python

Following : https://www.youtube.com/watch?v=T5pRlIbr6gg

Gender classifier :

In [8]:
import sys
print(sys.version)
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhshahane/body-measurements-dataset")

print("Path to dataset files:", path)

3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]


ModuleNotFoundError: No module named 'kagglehub'